# Elektroautos

Dein Chef hat die neuesten Daten über den Elektroautomarkt. Wie immer ist die Zeit knapp und um 14 Uhr hält dein Chef eine Presskonferenz über die neuen Zahlen. Da dein Chef selbst heute einen vollen Terminkalendar hat, bittet er dich die Daten auf- und für die Präsentation vorzubereiten. Dein Chef bekommt die Daten von dir um 13:50 Uhr und hat damit nur 10min Zeit deine Unterlagen durchzusehen.

Der Datensatz besteht aus zwei Teilen. Füge diese zunächst zusammen.

Erkunde den Datensatz. Hier kannst du beim Thema fehlende Werte weitere Vorgehensweisen ausprobieren, um die Daten zu befüllen. (Häufigkeiten bestimmen: **crosstab(),idxmax()** ...)

Weiterhin bietet der Datensatz ein neues **Features!** Erstelle dieses. (neue Informationen generieren aus den vorhandenen Daten)

Werte den Datensatz aus! Welche Erkenntnisse kannst du daraus gewinnen?
                            
Vergiss nicht dein Vorgehen zu dokumentieren: **Dokumentiere deine Schritte** - bringe deine Gedanken auf Papier!

### Wofür stehen die Spalten?


- Brand ... Manufacturer of the vehicle
- Model ... Model name
- AccelSec ... Acceleration as 0-100 km/h
- TopSpeed_KmH ... The top speed in km/h
- Rang_Km ... Range in km
- Efficiency_WhKm ... Efficiency Wh/km --> wie viel kw brauch ich pro km bzw. Stromverbrauch pro km
- FastCharge_KmH ... Charge km/h --> Reichweite in km nach einer Stunde Schnellladen (ist ein theoretischer Wert)
- RapidCharge ... Yes / No --> besitzt Schnellladefunktion ja/nein
- PowerTrain ... Front (FWD), rear (RWD) , or all wheel drive (AWD)
- PlugType ... Plug type (Stecktyp zum Laden an der Ladesäule; abhängig vom Land in dem das Auto später fährt)
- BodyStyle ... Basic size or style (SUV, Schrägdach etc.)
- Seats ... Number of seats
- PriceEuro ... Price in Germany before tax incentives


## to DO

                                       1. Data Preprocessing
                                           1.1) Datensatz einlesen
                                           1.2) fehlende Werte
                                           1.3) Ausreißer
                                           1.4) Speicherplatz
                                           1.5) Pairplot - erste visuelle Inspektion
                                           1.6) Features
                                        
                                        2. Visualisierung

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# Erste Datei einlesen
df_ecar1 = pd.read_csv(r'ecar_part1.csv', sep=',', index_col=0)
df_ecar1.head(3)


In [ ]:
# Zweite Datei einlesen
df_ecar2 = pd.read_csv(r'ecar_part2.csv', sep='|', index_col=0)
df_ecar2.head(3)


In [ ]:
# Dateien müssen Spaltenweise verbunden, 'connector' ist die KEY-Spalte
df_ecar = df_ecar1.merge(df_ecar2,on='connector')
df_ecar.reset_index
df_ecar

In [ ]:
# Speichern des aktuellen Standes des Datensatzes zur Dokumentation des Speicherplatzes
df_M00 = df_ecar

In [ ]:
df_M00.memory_usage(deep=True).sum() 

In [ ]:
# Spalte 'connector' löschen
df_ecar = df_ecar.drop(columns='connector')

In [ ]:
df_ecar.head(3)

# Fehlende Werte

In [ ]:
df_ecar.columns

In [ ]:
df_ecar.isnull().sum()

FastCharge_KmH hat Null Werte (5)

In [ ]:
df_ecar.head(3)

In [ ]:
df_ecar.dtypes

Datentypen 
Seats hat ? und sollte eine Zahl sein (Integer, da keine halben Sitze!)
Welche BodyStyles haben fehlende Werte




In [ ]:
# Kreuztabelle für Seats zum Bodystyle
pd.crosstab(df_ecar['Seats'], df_ecar['BodyStyle'], dropna=False)
#np.average((df_ecar['Seats'] '?'))

In [ ]:
# Welche BodyStyles haben fehlende Werte und welchen Durchschnitt bei Sitzen
for i in (df_ecar.loc[df_ecar['Seats']=='?','BodyStyle'].unique()):
    print(f'\nBodyStyle    : {i}')
    x = (pd.to_numeric((df_ecar.loc[df_ecar['BodyStyle']==i,'Seats']),errors='coerce')).mean()
    print(f'Durchschnitt : {x}\nWerte:')
    y = df_ecar.loc[df_ecar['BodyStyle']==i,'Seats']
    for i in y:
        print(i, end=' | ')
    
    

In [ ]:
# Prüfung welche Werte in den BodyStyles am häufigsten vorkommen
missing_types = df_ecar["BodyStyle"].loc[(df_ecar["Seats"] == '?')].unique()
for style in missing_types:
    print(style,'    \t: ', df_ecar.loc[(df_ecar['BodyStyle'] == style), 'Seats'].value_counts().idxmax(), ' Sitze')
    

In [ ]:
# Lösung von Beatrice ***************
missing_types = df_ecar["BodyStyle"].loc[(df_ecar["Seats"] == '?')].unique()
# am häufigsten auftretende Anzahl Sitze ermitteln mit idxmax() 
for style in missing_types:
    max_seats = df_ecar.loc[(df_ecar['BodyStyle'] == style), 'Seats'].value_counts().idxmax()
    print(style, max_seats)
    df_ecar.loc[(df_ecar["Seats"] == '?') & (df_ecar['BodyStyle'] == style),["Seats"]] = max_seats
df_ecar.head(-20)

In [ ]:
pd.crosstab((df_ecar['FastCharge_KmH'].isnull()) , df_ecar['RapidCharge'],dropna=False)

**FastCharge_KmH ist nur vorhanden wenn RapidCharge = 'Yes'**

Fehlende Werte können durch 0 ersetzt werden!

In [ ]:
# FastCharge 'NaN' in 0 ändern

df_ecar.loc[(df_ecar['FastCharge_KmH'].isnull()),["FastCharge_KmH"]] = 0
df_ecar.loc[(df_ecar['FastCharge_KmH'] == 0),:]

# Speicher optimieren 

In [ ]:
df_M01 = df_ecar
df_M01.memory_usage(deep=True)



In [ ]:
# Werte in 'FastCharge_KmH' als Integer setzen
df_ecar['FastCharge_KmH'] = pd.to_numeric(df_ecar.loc[:,'FastCharge_KmH'],downcast="integer")
df_ecar

In [ ]:
# Werte in 'Seats' als Integer setzen
df_ecar['Seats'] = pd.to_numeric(df_ecar.loc[:,'Seats'],downcast="integer")
df_ecar

In [ ]:
df_ecar[["TopSpeed_KmH","Range_Km","Efficiency_WhKm","PriceEuro"]] = df_ecar[["TopSpeed_KmH","Range_Km","Efficiency_WhKm","PriceEuro"]].apply(pd.to_numeric, downcast="integer")
df_ecar.dtypes

In [ ]:
#
df_ecar['AccelSec'] = pd.to_numeric(df_ecar.loc[:,'AccelSec'],downcast="float")
#df_ecar = df_ecar.drop(columns='SeAccelSecats')

In [ ]:
df_ecar.dtypes

In [ ]:
sns.pairplot(data=df_ecar)

In [ ]:
df_ecar.loc[df_ecar['TopSpeed_KmH'] > 300,:]


In [ ]:
df_ecar.memory_usage(deep=True).sum()

In [ ]:
# Speicherbedarf vor Optimierung
df_M00.memory_usage(deep=True).sum() 

In [ ]:
reduction = 100 * (df_M00.memory_usage(deep=True).sum()-df_ecar.memory_usage(deep=True).sum())/df_M00.memory_usage(deep=True).sum()
print(f"{reduction:0.2f}%")

In [ ]:
# Gruppe für PowerTrain anlegen
categories1 = df_ecar["PowerTrain"].unique()
print(f"Anzahl: {np.count_nonzero(categories1)} Werte: {categories1}")
categories2 = df_ecar["RapidCharge"].unique()
print(f"Anzahl: {np.count_nonzero(categories2)} Werte: {categories2}")
categories3 = df_ecar["PlugType"].unique()
print(f"Anzahl: {np.count_nonzero(categories3)} Werte: {categories3}")
categories4 = df_ecar["BodyStyle"].unique()
print(f"Anzahl: {np.count_nonzero(categories4)} Werte: {categories4}")
categories5 = df_ecar["Brand"].unique()
print(f"Anzahl: {np.count_nonzero(categories5)} Werte: {categories5}")

Werte in Spalte 'PowerTrain' und 'RapidCharge' können durch kürzere Strings ersetzt werden.

In [ ]:
# Kategorisieren der Spalten "PowerTrain" "RapidCharge" "PlugType" "BodyStyle"
df_ecar[["PowerTrain", "RapidCharge", "PlugType", "BodyStyle", "Brand"]] = df_ecar[["PowerTrain", "RapidCharge", "PlugType", "BodyStyle", "Brand"]].astype('category')
df_ecar.dtypes

In [ ]:
df_ecar.memory_usage(deep=True).sum()

In [ ]:
reduction = 100 * (df_M00.memory_usage(deep=True).sum()-df_ecar.memory_usage(deep=True).sum())/df_M00.memory_usage(deep=True).sum()
print(f"{reduction:0.2f}%")

In [ ]:
 df_ecar["Model"].unique()  # Macht keinen Sinn!

# Features
Zeit bis zur kompletten Aufladung wenn die Batterie leer ist: **Range_Km / FastCharge_KmH** (Achtung 0-Werte für FastCharge_KmH bei Fahrzeugen ohne RapidCharge!!)

Akkukapazität: **Range_Km * Efficiency_WhKm**



In [ ]:
# Full_Load = 'Range_Km' / 'FastCharge_KmH' : Zeit zur vollen Aufladung bei leerem Akku
df_ecar['Full_Load'] = round((df_ecar['Range_Km']/df_ecar['FastCharge_KmH'])*60,2)  # in 
#df_ecar.insert(8,'Full_Load',((df_ecar['Range_Km']/df_ecar.loc[df_ecar['FastCharge_KmH']>0,'FastCharge_KmH'])))
df_ecar

In [ ]:
# Cap_Acc = 'Range_Km' / 'Efficiency_WhKm' : Theoretische Kapazität des Akkus
#df_ecar.insert(9,'Cap_Acc',(df_ecar['Range_Km']*df_ecar['Efficiency_WhKm']))
df_ecar['Cap_Acc'] = ((df_ecar['Range_Km']) * (df_ecar['Efficiency_WhKm'] / 1000))


In [ ]:
df_ecar

In [ ]:
df_ecar[["AccelSec", "Full_Load", "Cap_Acc"]] = df_ecar[["AccelSec", "Full_Load", "Cap_Acc"]].apply(pd.to_numeric, downcast="float")
df_ecar.dtypes

In [ ]:
reduction = 100 * (df_M00.memory_usage(deep=True).sum()-df_ecar.memory_usage(deep=True).sum())/df_M00.memory_usage(deep=True).sum()
print(f"{reduction:0.2f}%")

In [ ]:
# alternativ: to_pickle Vorteil: Abspeicherung als DataFrame inkl. seiner Eigenschaften zum Speicherplatz
# Nachteil: wenn Versionen im Team nicht stimmen, kann der andere den DF nicht als pickle einlesen
df_ecar.to_pickle('ecar_after_preprocessing.pkl')

# Auswertungen


1. Vergleich der Hersteller mit ihren Modellen --> Wer ist denn am Markt? Gibt es Gloablplayer oder ist der Markt diversifiziert --> geeigneter Plot: Balkendiagramm (Säulendiagramm)
2. Autoeigenschaften vergleichen (Battery, Range_km, etc) --> Balkendiagramm, Scatterplot
3. Abhängikeiten vom Preis und Charkatereigenschaften --> Cut FUnktion --> Scatterplot
4. Ist RapidCharge eine special Funktion oder schon Standard? --> Piechart --> Können wir mit dieser Funktion werben oder nicht?
5. Wie lange brauche ich, um 1000 km zurückzulegen? --> stacked barchart (um unterscheiden zu können, wie viel der Zeit ich tanke und wie viel in der Zeit ich fahre)
6. Ist ein eauto nur für die Stadt geeigent oder kann ich auch ein eauto als Familienauto haben? --> Auswertung Bodytyle --> grouped bar chart


Hints: cut(), group by, agg(), Reihenfolge sortieren

Zu jedem Plot die Erkenntnisse aufschreiben. (Ausführlich mit Analyse und Erklärungen)
